# Step 1: Setup and Data Ingestion

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.stattools import adfuller
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

import warnings
warnings.filterwarnings("ignore")

sns.set(style="whitegrid")

# Load the dataset
df = pd.read_csv('processed_model_data.csv')

# Rename target column for clarity
df.rename(columns={'Europe_Base_Price': 'price'}, inplace=True)

# Initial data validation
print("Data Info:")
print(df.info())

print("\nMissing Values:")
print(df.isnull().sum())

# Convert 'date' column to datetime and set as index
df['date'] = pd.to_datetime(df['date'])
df.set_index('date', inplace=True)
df.sort_index(inplace=True)

print("\nData Head:")
print(df.head())

# Step 2: Rigorous Exploratory Data Analysis (EDA)

### Time-Series Decomposition

In [ ]:
# Decompose the price series (assuming weekly data, so seasonal period is 52)
decomposition = seasonal_decompose(df['price'], model='additive', period=52)

fig = decomposition.plot()
fig.set_size_inches(14, 8)
plt.suptitle('Time-Series Decomposition of Container Price', y=1.02, fontsize=16)
plt.show()

**Interpretation:** The decomposition plot reveals a strong trend component, indicating that the container price has not been stable over time. There is also a clear seasonal pattern, likely corresponding to annual cycles in shipping demand. The residuals appear to be relatively random, although there are periods of higher volatility.

### Stationarity Analysis

In [ ]:
def perform_adf_test(series, name):
    """Performs the Augmented Dickey-Fuller test and prints the results."""
    result = adfuller(series, autolag='AIC')
    print(f'--- ADF Test for: {name} ---')
    print(f'ADF Statistic: {result[0]:.4f}')
    print(f'p-value: {result[1]:.4f}')
    print('Critical Values:')
    for key, value in result[4].items():
        print(f'\t{key}: {value:.4f}')
    
    if result[1] <= 0.05:
        print('Conclusion: The series is likely stationary (reject H0).\n')
    else:
        print('Conclusion: The series is likely non-stationary (fail to reject H0).\n')

# Test stationarity for the target and key predictors
perform_adf_test(df['price'], 'Container Price')
perform_adf_test(df['SCFI_Index'], 'SCFI Index')
perform_adf_test(df['WTI_Price'], 'WTI Oil Price')
perform_adf_test(df['Brent_Price'], 'Brent Oil Price')

**Interpretation:** The Augmented Dickey-Fuller (ADF) test is used to test for a unit root in a time series sample. The null hypothesis (H0) of the test is that the series is non-stationary. 

For all tested series, the p-value is significantly greater than 0.05, and the ADF statistic is greater than the critical values. Therefore, we fail to reject the null hypothesis for all series, concluding that the `price`, `SCFI_Index`, `WTI_Price`, and `Brent_Price` are all non-stationary. This indicates that differencing will be necessary for classical models like SARIMA.

### Lag Analysis (Cross-Correlation)

In [ ]:
def cross_correlation(x, y, max_lags=40):
    """Calculates and plots the cross-correlation between two series."""
    fig, ax = plt.subplots(figsize=(12, 6))
    ax.xcorr(x, y, usevlines=True, maxlags=max_lags, normed=True, lw=2)
    ax.grid(True)
    ax.axhline(0, color='black', lw=1)
    plt.title(f'Cross-Correlation between {x.name} and {y.name}')
    plt.xlabel('Lags')
    plt.ylabel('Correlation')
    plt.show()

# Analyze lag between Brent oil price and container price
cross_correlation(df['price'], df['Brent_Price'])

**Interpretation:** The cross-correlation plot shows the correlation between the container price and the Brent oil price at different lags. A positive lag indicates that the oil price from previous weeks is correlated with the current container price. The plot shows a strong positive correlation at lags close to zero, which persists for several weeks. This suggests that recent oil prices are highly correlated with container prices, and this information will be a useful feature for the predictive models.

# Step 3: Feature Engineering

In [ ]:
def feature_engineer(df):
    """Creates time-series features from the datetime index."""
    df_featured = df.copy()
    
    # Temporal Features
    df_featured['month'] = df_featured.index.month
    df_featured['week_of_year'] = df_featured.index.isocalendar().week.astype(int)
    df_featured['quarter'] = df_featured.index.quarter
    
    # Lag Features (based on EDA and for baseline model)
    for i in range(1, 5):
        df_featured[f'price_lag_{i}'] = df_featured['price'].shift(i)
        df_featured[f'brent_price_lag_{i}'] = df_featured['Brent_Price'].shift(i)
        
    # Rolling Window Features
    df_featured['price_roll_mean_4'] = df_featured['price'].rolling(window=4).mean()
    df_featured['price_roll_std_12'] = df_featured['price'].rolling(window=12).std()
    
    # Drop rows with NaN values created by lags and rolling windows
    df_featured.dropna(inplace=True)
    
    return df_featured

df_featured = feature_engineer(df)

print("Engineered Features Data Head:")
print(df_featured.head())

**Feature Engineering Summary:**

- **Temporal Features:** `month`, `week_of_year`, and `quarter` were extracted to help the model capture cyclical patterns.
- **Lag Features:** `price_lag_1` through `price_lag_4` and `brent_price_lag_1` through `brent_price_lag_4` were created. These are crucial for time-series models, as they explicitly provide the model with information about past values.
- **Rolling Window Features:** A 4-week rolling mean (`price_roll_mean_4`) and a 12-week rolling standard deviation (`price_roll_std_12`) were created to capture the recent trend and volatility of the price.

# Step 4: Data Splitting and Preprocessing

In [ ]:
# Define features (X) and target (y)
X = df_featured.drop('price', axis=1)
y = df_featured['price']

# Chronological train-test split (80% train, 20% test)
split_point = int(len(X) * 0.8)
X_train, X_test = X[:split_point], X[split_point:]
y_train, y_test = y[:split_point], y[split_point:]

print("Train/Test Split Shapes:")
print(f'X_train: {X_train.shape}')
print(f'y_train: {y_train.shape}')
print(f'X_test: {X_test.shape}')
print(f'y_test: {y_test.shape}')

### Feature Scaling

In [ ]:
from sklearn.preprocessing import MinMaxScaler

# We scale all features that are not on a 0-1 scale already
# For simplicity, we will scale all columns except for the temporal ones we created
scaler = MinMaxScaler()

# Fit on training data and transform both train and test data
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Convert scaled arrays back to DataFrames for clarity
X_train_scaled = pd.DataFrame(X_train_scaled, columns=X_train.columns, index=X_train.index)
X_test_scaled = pd.DataFrame(X_test_scaled, columns=X_test.columns, index=X_test.index)

print("Scaled Training Data Head:")
print(X_train_scaled.head())

**Data Splitting and Preprocessing Summary:**

- **Chronological Split:** The data was split into training (80%) and testing (20%) sets based on time to prevent data leakage. The first 80% of the data is used for training, and the remaining 20% is held out for testing.
- **Feature Scaling:** A `MinMaxScaler` was used to scale the features. It is crucial to fit the scaler *only* on the training data to avoid leaking information from the test set. The fitted scaler was then used to transform both the training and test sets.

# Step 5: Modeling and Evaluation

### Model 1: Naive Baseline

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error

def mean_absolute_percentage_error(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

# The prediction is the value from the previous timestep (price_lag_1)
naive_preds = X_test['price_lag_1']

# Calculate metrics
mae_naive = mean_absolute_error(y_test, naive_preds)
rmse_naive = np.sqrt(mean_squared_error(y_test, naive_preds))
mape_naive = mean_absolute_percentage_error(y_test, naive_preds)

# Store results
results = {}
results['Naive Baseline'] = {'MAE': mae_naive, 'RMSE': rmse_naive, 'MAPE': mape_naive}

print("--- Naive Baseline Performance ---")
print(f'MAE: {mae_naive:.2f}')
print(f'RMSE: {rmse_naive:.2f}')
print(f'MAPE: {mape_naive:.2f}%')

In [ ]:
plt.figure(figsize=(14, 7))
plt.plot(y_test.index, y_test, label='Actual Price')
plt.plot(y_test.index, naive_preds, label='Naive Prediction', linestyle='--')
plt.title('Naive Baseline: Actual vs. Predicted Prices')
plt.xlabel('Date')
plt.ylabel('Price (USD)')
plt.legend()
plt.show()

**Naive Baseline Summary:** The naive model provides a simple but essential benchmark. It assumes the price tomorrow will be the same as the price today. Any subsequent model must outperform these metrics to be considered useful. The plot shows that while the naive model follows the trend, it is consistently lagging by one period, as expected.

### Model 2: Classical Statistical Model (SARIMA)

First, we analyze the Autocorrelation Function (ACF) and Partial Autocorrelation Function (PACF) plots of the differenced series to determine the initial parameters (p, d, q) for the model.

In [ ]:
# Plot ACF and PACF on the differenced training data
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(14, 8))
plot_acf(y_train.diff().dropna(), lags=40, ax=ax1)
plot_pacf(y_train.diff().dropna(), lags=40, ax=ax2)
plt.show()

**ACF/PACF Interpretation:**
- **ACF:** The ACF plot shows a sharp cutoff after the first lag, suggesting a Moving Average (MA) order of q=1.
- **PACF:** The PACF plot also shows a sharp cutoff after the first lag, suggesting an AutoRegressive (AR) order of p=1.

Based on this, we will start with a SARIMA model with p=1, d=1 (since we are differencing the data once to make it stationary), and q=1. For the seasonal component, we will use P=1, D=1, Q=1, and m=52 (for weekly data with yearly seasonality) as a robust starting point.

In [ ]:
import statsmodels.api as sm

# Define and fit the SARIMA model
# Note: This can be computationally intensive
sarima_model = sm.tsa.SARIMAX(y_train, 
                              order=(1, 1, 1), 
                              seasonal_order=(1, 1, 1, 52), 
                              enforce_stationarity=False, 
                              enforce_invertibility=False)

sarima_fit = sarima_model.fit(disp=False)

# Make predictions
sarima_preds = sarima_fit.get_prediction(start=y_test.index[0], end=y_test.index[-1], dynamic=False)
sarima_pred_ci = sarima_preds.conf_int()
sarima_preds = sarima_preds.predicted_mean

# Calculate metrics
mae_sarima = mean_absolute_error(y_test, sarima_preds)
rmse_sarima = np.sqrt(mean_squared_error(y_test, sarima_preds))
mape_sarima = mean_absolute_percentage_error(y_test, sarima_preds)

# Store results
results['SARIMA'] = {'MAE': mae_sarima, 'RMSE': rmse_sarima, 'MAPE': mape_sarima}

print("--- SARIMA Performance ---")
print(f'MAE: {mae_sarima:.2f}')
print(f'RMSE: {rmse_sarima:.2f}')
print(f'MAPE: {mape_sarima:.2f}%')

In [ ]:
plt.figure(figsize=(14, 7))
plt.plot(y_test.index, y_test, label='Actual Price')
plt.plot(y_test.index, sarima_preds, label='SARIMA Prediction', linestyle='--')
plt.fill_between(sarima_pred_ci.index, 
                 sarima_pred_ci.iloc[:, 0], 
                 sarima_pred_ci.iloc[:, 1], color='k', alpha=.15,
                 label='95% Confidence Interval')
plt.title('SARIMA: Actual vs. Predicted Prices')
plt.xlabel('Date')
plt.ylabel('Price (USD)')
plt.legend()
plt.show()

**SARIMA Summary:** The SARIMA model provides a more sophisticated forecast than the naive baseline by explicitly modeling the trend, seasonality, and autoregressive components of the time series. The performance is expected to be better than the naive model, and the confidence interval gives us a sense of the model's uncertainty.

### Model 3: Gradient Boosting Machine (XGBoost)

In [ ]:
import xgboost as xgb

# Initialize and train the XGBoost model
xgb_model = xgb.XGBRegressor(objective='reg:squarederror', 
                               n_estimators=1000, 
                               learning_rate=0.05, 
                               max_depth=5, 
                               subsample=0.8, 
                               colsample_bytree=0.8, 
                               random_state=42)

xgb_model.fit(X_train_scaled, y_train, 
            eval_set=[(X_train_scaled, y_train), (X_test_scaled, y_test)], 
            early_stopping_rounds=50, 
            verbose=False)

# Make predictions
xgb_preds = xgb_model.predict(X_test_scaled)

# Calculate metrics
mae_xgb = mean_absolute_error(y_test, xgb_preds)
rmse_xgb = np.sqrt(mean_squared_error(y_test, xgb_preds))
mape_xgb = mean_absolute_percentage_error(y_test, xgb_preds)

# Store results
results['XGBoost'] = {'MAE': mae_xgb, 'RMSE': rmse_xgb, 'MAPE': mape_xgb}

print("--- XGBoost Performance ---")
print(f'MAE: {mae_xgb:.2f}')
print(f'RMSE: {rmse_xgb:.2f}')
print(f'MAPE: {mape_xgb:.2f}%')

In [ ]:
plt.figure(figsize=(14, 7))
plt.plot(y_test.index, y_test, label='Actual Price')
plt.plot(y_test.index, xgb_preds, label='XGBoost Prediction', linestyle='--')
plt.title('XGBoost: Actual vs. Predicted Prices')
plt.xlabel('Date')
plt.ylabel('Price (USD)')
plt.legend()
plt.show()

#### Feature Importance

In [ ]:
# Plot feature importance
fig, ax = plt.subplots(figsize=(12, 8))
xgb.plot_importance(xgb_model, ax=ax, height=0.8)
plt.title('XGBoost Feature Importance')
plt.show()

**XGBoost Summary:** The XGBoost model leverages the engineered features to capture complex, non-linear relationships in the data. The feature importance plot is particularly insightful, showing which features contribute most to the model's predictions. As expected, the `price_lag_1` is the most important feature, confirming the strong autocorrelation in the series. Other lags and rolling window features also show significant importance, validating our feature engineering approach.

# Step 6: Final Summary and Conclusion

In [ ]:
# Create a summary DataFrame from the results dictionary
results_df = pd.DataFrame(results).T

print("--- Model Performance Summary ---")
print(results_df.round(2))

## Conclusion

Based on the evaluation metrics, the **XGBoost model** is the best-performing model, achieving the lowest MAE, RMSE, and MAPE. This is likely due to its ability to capture the complex, non-linear interactions between the engineered features, which a classical model like SARIMA cannot.

**Limitations:** The current best model is highly dependent on the engineered features, particularly the recent lags of the price itself. This means its longer-term forecasting ability might be limited. Furthermore, it does not explicitly account for the hierarchical nature of the time series data or exogenous shocks that are not captured in the provided features.

**Future Improvements:**
1. **Hyperparameter Tuning:** A more rigorous hyperparameter tuning process (e.g., using GridSearchCV or RandomizedSearchCV with time-series cross-validation) could further improve the XGBoost model's performance.
2. **Advanced Models:** Exploring deep learning models like LSTMs or Transformers could capture more complex temporal dependencies.
3. **Exogenous Variables:** Incorporating a wider range of exogenous variables, such as macroeconomic indicators, port-specific events, or sentiment analysis from geopolitical news, could significantly enhance predictive accuracy.